In [153]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from IPython.display import Audio, display
from ipywidgets import interact

%matplotlib inline

In [171]:
def discrete_normal_distribution(frequency, num_harmonics, mean=None):
    array = np.arange(1, num_harmonics + 1, dtype=np.float32)
    mean  = np.median(array) if mean is None else mean
    coefficients = stats.norm(mean, np.std(array)*2).pdf(array)
    coefficients /= sum(coefficients)
    harmonics = array * frequency
    return (harmonics, coefficients)

def generate_waveform(harmonics, fourier_coeffs, sample_rate=44100, duration=3):
    length = int(sample_rate * duration)
    discrete_times = np.linspace(0, duration, length, endpoint=False)
    waveform = np.zeros(length)
    
    for freq, coeff in zip(harmonics, fourier_coeffs):
        waveform += (np.sin(2 * np.pi * freq * discrete_times) * coeff)
    # Normalize the waveform between -1 and 1
    waveform /= np.max(np.abs(waveform))
    return np.int16(waveform * 32767)

In [172]:
sample_rate = 44100
duration = 3

@interact(fun=(50, 500, 10), num_harms=(4, 20, 1), h_mean=(1, 10, 1))
def plot_spectrum(fun=100, num_harms=10, h_mean=5):

    harmonics, coefs = discrete_normal_distribution(frequency=fun, num_harmonics=num_harms, mean=h_mean)
    waveform = generate_waveform(harmonics, coefs, duration = duration, sample_rate=sample_rate)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4)) 
    fig.suptitle(f"""{fun} Hz harmonic complex with {num_harms}""", weight="bold")
    fig.subplots_adjust(wspace=0.4, top=0.85)

    ax1.bar(harmonics, coefs, width=0.5*fun)
    ax1.set_xlabel("Array")
    ax1.set_ylabel("Coefficient")
    ax1.set_title(f"mean = {h_mean * fun} Hz, std = {np.std(harmonics):.2f}")

    n_period = 3
    periods = n_period / fun
    num_sample = int(periods * sample_rate)
    time = np.linspace(0, periods, num_sample)

    ax2.plot(time, waveform[:num_sample])
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Amplitude")
    ax2.set_title("Waveform")

    display(Audio(waveform, rate=sample_rate))



interactive(children=(IntSlider(value=100, description='fun', max=500, min=50, step=10), IntSlider(value=10, d…